# Dependencies

In [10]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import classification_report

# Cleaning up Training Data

In [11]:
def clean_data(df):        
    # Dropping columns that are not needed
    df = df.drop(['Pos'], axis=1)

    # Scaling the data
    Scaler = MinMaxScaler()
    df = pd.DataFrame(Scaler.fit_transform(df), columns=df.columns)
    
    return df

In [12]:
df_train = pd.read_csv('Data/Training/overall_training_data.csv',index_col=0)
df_train = df_train.dropna(axis=0)
df_train = df_train.drop(['Player'], axis=1)
df_train = clean_data(df_train)

# Training the Model

In [13]:
training_cols = [x for x in df_train.columns if x != 'All Star']
X_train = df_train[training_cols]
y_train = df_train['All Star']

SVC_Model = SVC(kernel='rbf', gamma=1.0,C=10.0)
SVC_Model = SVC_Model.fit(X_train,y_train)
y_train_prediction = SVC_Model.predict(X_train)

In [14]:
# Accuracy of model on training data using sklearn.metrics
print(metrics.accuracy_score(y_train, y_train_prediction))

0.981974921630094


# Running Model on Test Data

In [15]:
# Processing test data
df_test = pd.read_csv('Data/Testing/Test_Data.csv', index_col=0)
df_test = df_test.dropna(axis=0)
player_list = df_test['Player']
df_test = df_test.drop(['Player'], axis=1)
df_test = clean_data(df_test)

# Predict using the LR Model
y_prediction_test = SVC_Model.predict(df_test)

In [16]:
# Creating dataframe
result = pd.DataFrame(player_list)
result['Predicted All Star'] = y_prediction_test.tolist()

# Actual NBA all stars
actual_all_stars = ['Bradley Beal', 'Kyrie Irving', 'Giannis Antetokounmpo', 'Joel Embiid', 'Jaylen Brown', 
 'James Harden', 'Zach LaVine', 'Ben Simmons', 'Julius Randle', 'Jayson Tatum', 'Nikola Vučević',
 'Stephen Curry', 'Luka Dončić', 'LeBron James', 'Kawhi Leonard', 'Nikola Jokić', 'Anthony Davis', 'Damian Lillard', 
 'Donovan Mitchell', 'Chris Paul', 'Paul George', 'Zion Williamson', 'Rudy Gobert', 'Kevin Durant','Domantas Sabonis',
                   'Jimmy Butler']

result['Actual All Star'] = 0.0
for index, row in result.iterrows():
    if result.at[index, 'Player'] in actual_all_stars:
        result.at[index, 'Actual All Star'] = 1.0
result

,Player,Predicted All Star,Actual All Star
0,Precious Achiuwa,0.0,0.0
1,Jaylen Adams,0.0,0.0
2,Steven Adams,0.0,0.0
3,Bam Adebayo,1.0,0.0
4,LaMarcus Aldridge,0.0,0.0
...,...,...,...
540,Delon Wright,0.0,0.0
541,Thaddeus Young,0.0,0.0
542,Trae Young,0.0,0.0
543,Cody Zeller,0.0,0.0


In [17]:
print(classification_report(result['Actual All Star'], result['Predicted All Star']))

              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98       515
         1.0       0.67      0.21      0.32        28

    accuracy                           0.95       543
   macro avg       0.81      0.60      0.65       543
weighted avg       0.94      0.95      0.94       543

